# Notebook to compute the specialization for all seeds of a given experiment

* Set the correct number of landmarks
* Set the number of agents in the population 
* Set the results dir containing all the seeds to report

In [1]:
import json
import os

import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations
import pandas as pd

In [8]:
NUMBER_OF_LANDMARKS = 3
n_agents = 4
results_dir = "/home/elias/paper/population/4/communication"


individual_goals = np.eye(NUMBER_OF_LANDMARKS, dtype=np.uint8).tolist()
collective_goals = np.array(list(combinations(individual_goals, 2))).sum(1).tolist()
goals = ["".join(str(t) for t in g) for g in collective_goals]
goals_index = {i: g for i, g in zip(range(len(goals)), goals)}
agents = [f"agent_{i}" for i in range(n_agents)]

In [9]:
specializations_during_training = []
specializations_convergence = []
for subdir in os.listdir(results_dir):
    full_path = os.path.join(results_dir, subdir, "result.json")
    result_dump = open(full_path, "r")
    # parse metrics
    for result in result_dump:
        # we always redefine this to only consider the last set of results (last evaluation)
        metrics = json.loads(result).get("evaluation")
        if metrics is not None:
            for g in goals:
                for agent in agents:
                    if (
                        metrics["hist_stats"].get(f"{agent} position for {g}")
                        is not None
                        and len(metrics["hist_stats"].get(f"{agent} position for {g}"))
                        > 0
                    ):
                        aux = pd.DataFrame(
                            metrics["hist_stats"].get(f"{agent} position for {g}")
                        ).value_counts()
                        specializations_during_training.append(
                            aux.value_counts().max() / aux.value_counts().sum()
                        )
    specializations_convergence.extend(
        specializations_during_training[-len(goals) * n_agents :]
    )

In [10]:
print(
    f"Specialization at convergence {sum(specializations_convergence) / len(specializations_convergence)}"
)
print(
    f"Specialization during training {sum(specializations_during_training) / len(specializations_during_training)}"
)

Specialization at convergence 0.7083333333333334
Specialization during training 0.7007278835386338
